Escape velocity
https://byjus.com/physics/derivation-of-escape-velocity/

 With the help of the escape velocity formula, it is possible to calculate the minimum velocity that an object requires to overcome a particular planet’s or object’s gravitational pull
 
 v_e =  sqrt(2 G M /r) 

Assume a perfect sphere-shaped planet of radius R and mass M. Now, if a body of mass m is projected from a point A on the surface of the planet. 

In [1]:
using DynamicPolynomials
using JuMP
using MosekTools
using LinearAlgebra
using CSV
using DataFrames
import JSON
using Dates
using Gurobi

functions for generating monomials

In [2]:
function all_monomials_up_to_max_deg(x, deg)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:deg 
      for m=all_monomials_up_to_max_deg(x[2:end], deg)
    ]
    end
end

function mons_of_max_degree_and_unit(x, deg, u)
    [m
        for m=all_monomials_up_to_max_deg(x, deg)
        #if all(unit(m) .== u)
    ]
end
                
function degree_poly(p)
    maximum(degree.(monomials(p)))
end

function all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall) 
                for m=all_monomials_up_to_max_deg_overall(x[2:end], deg, deg_overall-k)
    ]
    end
end

function mons_of_max_degree_and_unit_overall(x, deg, deg_overall, u)
    [m
        for m=all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
        #if all(unit(m) .== u)
    ]
end

function all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall, theDegrees[1]) 
                for m=all_monomials_up_to_max_deg_overall_and_individual(x[2:end], deg, deg_overall-k, theDegrees[2:end])
    ]
    end
end

all_monomials_up_to_max_deg_overall_and_individual (generic function with 1 method)

In [3]:
@polyvar m v_e r E_k_i E_k_f U_g_i U_g_f G M # the order matters per evaluation(?) # a_y a_x dv_x dv_y

x =     [m v_e r E_k_i E_k_f U_g_i U_g_f G M ] #dv_x dv_y a_y a_x

1×9 Matrix{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 m  v_e  r  E_k_i  E_k_f  U_g_i  U_g_f  G  M

In [8]:

#dx2dt2=differentiate(differentiate(dx,dt),dt)
#dy2dt2=differentiate(differentiate(dy,dt),dt)

0

In [4]:
axioms= 
[
    E_k_i - 1/2 * m * v_e^2,            #  initial kinetic energy   
    E_k_f - 0,                          # final kinetic energy is 0
    U_g_i *r + G *M * m,                # initial garvitational potential
    U_g_f - 0,                          # final energy 
    E_k_i + U_g_i - E_k_f - U_g_f,      # conservation of energy
    #F * dx^2 - G * M *m,               # the gravitational force
    #dW - F * dx,                        # work needed to displace a body by dx distance against the gravitational force pull
    #work done against the gravitational attraction to take the body from the planet’s surface to infinity can be easily calculated by integrating the equation for work done within the limits x = R to x = ∞
    #m*dx2dt2*r^3-G*M*m*dx,               # force in the x direction
    #m*dy2dt2*r^3-G*M*m*dy,               # force in the y direction
    #m*a_x*r^3-G*M*m*dx,                  # force in the x direction
    #m*a_y*r^3-G*M*m*dy,                  # force in the y direction
  #  r^2 - dx^2 - dy^2,                    # polar coordinates transformation
    #dx2dt2 * r^2 +  G*M *r,                    # gravitational field intensity of point mass
    #E_g_x * m - m * dx2dt2,
    #E_g_y * m - M * dy2dt2,
]
    #a_x* dt - dv_x,                    # a_x = dv_x / dt
    #a_y* dt - dv_y,                    # a_y = dv_y / dt
    

5-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 E_k_i - 0.5mv_e²
 E_k_f
 rU_g_i + mGM
 U_g_f
 -U_g_f + U_g_i - E_k_f + E_k_i

In [5]:
deg = 3
deg_overall = 8
theDegrees = [1  2  3 2  2  2      2     2    1 1] 
            # m v_e r E_k_i E_k_f U_g_i U_g_f G M

candidate_mons = [
    #mons_of_max_degree_and_unit_overall(params, deg, deg_overall, [])
    all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    for ai=axioms
]

@show size.(candidate_mons)

# model = Model(Mosek.Optimizer)
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 600.0)

mons_q = mons_of_max_degree_and_unit_overall([G M m v_e r], deg, deg_overall, []) #  only include variables expected to appear in the final theorem 
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
eqs = coefficients(residual)

# Ensure that the sum of the coefficients on the terms involving m isn't zero, in order that m is part of expression
@constraint model sum(coeff_q[degree.(mons_q, [v_e ]).>0]) == 1.0

@constraint model eqs .== 0
@objective model Max 0

optimize!(model)
@show termination_status(model)

size.(candidate_mons) = [(5832,), (5832,), (5832,), (5832,), (5832,)]
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-04
Set parameter TimeLimit to value 600
size.(coeff_αs) = [(5832,), (5832,), (5832,), (5832,), (5832,)]
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 24717 rows, 29827 columns and 59433 nonzeros
Model fingerprint: 0xfe2d1c27
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 24717 rows and 29827 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00

OPTIMAL::TerminationStatusCode = 1

In [6]:
value_poly = p -> sum(value.(coefficients(p)).* monomials(p))

#29 (generic function with 1 method)

In [7]:
value_q = value_poly(q)
value_q

-2.0mGM + mv_e²r

Rearanging, we get the expected expression for the escape velocity : v_e^2 = 2G M / r
that is v_e = \sqrt(2G M / r)

In [8]:
@show value_αs = value_poly.(αs)

value_αs = value_poly.(αs) = Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}[-2.0r, 2.0r, -2.0, 2.0r, 2.0r]


5-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 -2.0r
 2.0r
 -2.0
 2.0r
 2.0r